In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import os

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import cv2
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [3]:
 ! pip install -q kaggle

In [4]:
from google.colab import files

In [5]:
! kaggle datasets download -d darshbaxi/doodle

Dataset URL: https://www.kaggle.com/datasets/darshbaxi/doodle
License(s): unknown
 98% 118M/121M [00:01<00:00, 113MB/s] 
100% 121M/121M [00:01<00:00, 97.6MB/s]


In [6]:

!unzip doodle.zip

Streaming output truncated to the last 5000 lines.
  inflating: Doodle_recog/square/5245595549696000.png  
  inflating: Doodle_recog/square/5246157921976320.png  
  inflating: Doodle_recog/square/5246415955558400.png  
  inflating: Doodle_recog/square/5246549435088896.png  
  inflating: Doodle_recog/square/5246639885254656.png  
  inflating: Doodle_recog/square/5248218063437824.png  
  inflating: Doodle_recog/square/5249312625786880.png  
  inflating: Doodle_recog/square/5249507392487424.png  
  inflating: Doodle_recog/square/5249857688174592.png  
  inflating: Doodle_recog/square/5250768724557824.png  
  inflating: Doodle_recog/square/5250877352837120.png  
  inflating: Doodle_recog/square/5251851085676544.png  
  inflating: Doodle_recog/square/5252495536291840.png  
  inflating: Doodle_recog/square/5253748051935232.png  
  inflating: Doodle_recog/square/5253905292197888.png  
  inflating: Doodle_recog/square/5255104225607680.png  
  inflating: Doodle_recog/square/5255288133255168.png

In [7]:
train_dataset_path = '/content/Doodle_recog'

In [8]:
IMG_WIDTH = 255
IMG_HEIGHT = 255
BATCH_SIZE = 32

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data generator for training with validation split
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest',
                                   validation_split=0.2)  # 20% of the data will be used for validation

# Training data generator
train_generator = train_datagen.flow_from_directory(train_dataset_path,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    subset='training')  # use 'training' subset

# Validation data generator
validation_generator = train_datagen.flow_from_directory(train_dataset_path,
                                                         target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                         batch_size=BATCH_SIZE,
                                                         class_mode='categorical',
                                                         shuffle=True,
                                                         subset='validation')  # use 'validation' subset




Found 24000 images belonging to 9 classes.
Found 6000 images belonging to 9 classes.


In [ ]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for classes present in the training and validation datasets

0 : baseball
1 : bread
2 : circle
3 : hexagon
4 : moon
5 : none
6 : octagon
7 : square
8 : star


In [ ]:
def create_model():
    model = Sequential([
        Conv2D(filters=128, kernel_size=(5, 5), padding='valid', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Flatten(),

        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=9, activation='softmax')
    ])

    return model

In [ ]:
cnn_model = create_model()

In [ ]:
print(cnn_model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 251, 251, 128)       │           9,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 251, 251, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 125, 125, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 125, 125, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 123, 123, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 123, 123, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 61, 61, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 59, 59, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 59, 59, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 29, 29, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 29, 29, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 26912)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,889,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 9)                   │           2,313 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,994,921 (26.68 MB)

 Trainable params: 6,994,473 (26.68 MB)

 Non-trainable params: 448 (1.75 KB)

None


In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)

In [ ]:
optimizer = Adam(learning_rate=0.001)

In [ ]:
cnn_model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = cnn_model.fit(train_generator, epochs=50, validation_data=validation_generator,
                       verbose=2,
                       callbacks=[reduce_lr])

Epoch 1/50
750/750 - 475s - 634ms/step - accuracy: 0.4694 - loss: 1.6870 - val_accuracy: 0.2000 - val_loss: 5.4079 - learning_rate: 0.0010
Epoch 2/50
750/750 - 478s - 637ms/step - accuracy: 0.5787 - loss: 1.1981 - val_accuracy: 0.2000 - val_loss: 3.4687 - learning_rate: 0.0010
Epoch 3/50
750/750 - 449s - 599ms/step - accuracy: 0.6195 - loss: 1.0814 - val_accuracy: 0.2000 - val_loss: 2.5136 - learning_rate: 0.0010
Epoch 4/50
750/750 - 450s - 599ms/step - accuracy: 0.6392 - loss: 1.0307 - val_accuracy: 0.1998 - val_loss: 4.2952 - learning_rate: 0.0010
Epoch 5/50
750/750 - 448s - 597ms/step - accuracy: 0.6608 - loss: 0.9749 - val_accuracy: 0.2002 - val_loss: 4.4285 - learning_rate: 0.0010
Epoch 6/50
750/750 - 446s - 595ms/step - accuracy: 0.6715 - loss: 0.9461 - val_accuracy: 0.2367 - val_loss: 12.9468 - learning_rate: 0.0010
Epoch 7/50
750/750 - 448s - 598ms/step - accuracy: 0.6794 - loss: 0.9210 - val_accuracy: 0.1972 - val_loss: 2.8625 - learning_rate: 0.0010
Epoch 8/50
750/750 - 560s 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = cnn_model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred_classes)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()


In [ ]:
cnn_model.save('my_model.h5')

In [ ]:
import pickle

with open('my_model.pkl', 'wb') as file:
    pickle.dump(history.history, file)

